In [1]:
import copy
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, DynamicCache, StaticCache
from peft import PeftModel
torch.cuda.empty_cache()
model_name = "meta-llama/Llama-2-7b-hf"
lora_name = ["iamshnoo/alpaca-2-7b-english","iamshnoo/alpaca-2-7b-chinese","jb-01/llama-2-7b-ai2-arc"]
model_path = "/home/featurize/work/Lorac_recom/models/" 
lora_path= ["/home/featurize/work/Lorac_recom/lora_models/lora1",
            "/home/featurize/work/Lorac_recom/lora_models/lora2",
            "/home/featurize/work/Lorac_recom/lora_models/lora3"]

/home/featurize/work/lorac/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def B_inverse_weight(model,num_layers):
    W_b=[]
    W_b_inver =[]
    state_dict = model.state_dict()
    for i in range(num_layers):
        layer_key = f'base_model.model.model.layers.{i}.self_attn.v_proj.lora_B.default.weight'
        layer_B = state_dict[layer_key]
        W_b.append(layer_B)
        layer_B_inverse = torch.linalg.pinv(layer_B)
        W_b_inver.append(layer_B_inverse)
    return W_b,W_b_inver
def get_v_cache(kv_cache):
    v_cache=[]
    for layer_idx, (key, value) in enumerate(kv_cache):
        v_cache.append(value.permute(0, 1, 3, 2).reshape(1, 4096, -1))
    return v_cache
def compute_miniKV(main_KV, merged_KV, B_inverse_weight):
    mini_Vs=[]
    merged_V = get_v_cache(merged_KV)
    main_V = get_v_cache(main_KV)
    res_V = [merged_V[i]- main_V[i] for i in range(len(merged_V))]
    # print(len(main_V),main_V[0].shape)
    # print(len(merged_V),merged_V[0].shape)
    for i in range(len(res_V)):
        # print(res_v[i].dtype)
        mini_V = torch.matmul(B_inverse_weight[i].to(torch.float32),res_V[i]) 
        mini_Vs.append(mini_V)
    return mini_Vs
def recompute(kv_cache,W_B):
    re_caches=[]
    for i in range(len(kv_cache)):
        re_cache=torch.matmul(W_B[i].to(torch.float32),kv_cache[i]).view(1,32,128,-1).permute(0, 1, 3, 2) #1,64,4    4096,64 = [1,4096,4]   -> [1, 32, 4, 128]
        re_caches.append(re_cache)
    return re_caches
def tuple_to_list(kv_tuple):
    main_kv_list = [list(item) for item in kv_tuple]
    return main_kv_list

def list_to_tuple(kv_list):
    main_kv_tuple =tuple(tuple(item) for item in kv_list)
    return main_kv_tuple
def get_re_mergeKV(W_B,kv_cache,main_kv):
    re_v_cache = recompute(kv_cache,W_B)
    main_kv = tuple_to_list(main_kv)
    for i in range(len(main_kv)):
        main_kv[i][1]+=re_v_cache[i]
    main_kv = list_to_tuple(main_kv)
    main_kv = DynamicCache.from_legacy_cache(main_kv)
    return main_kv

In [3]:
torch.cuda.empty_cache()

In [ ]:
SYSTEM_PROMPT="You are a helpful assistant. "

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float32, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards: 100%|██████████| 3/3 [01:02<00:00, 20.93s/it]


In [5]:
SYSTEM_PROMPT=""" Donald Trump’s pick for defense secretary, was defiant Wednesday in the face of doubts that he can win confirmation as he met with Republican lawmakers and sought to address concerns about a series of misconduct allegations.

Hegseth vowed to “fight like hell” to win confirmation — and told at least one GOP senator he would quit drinking alcohol if he gets the job.

The veteran and former Fox News host’s bid to overcome questions among some Republicans comes amid reports that Trump’s team is already considering other options to lead the Pentagon.

Hegseth said he spoke to Trump on Wednesday and still has the president-elect’s support.

In an interview with conservative radio host Megyn Kelly, Hegseth said that Trump told him, “Hey Pete, I got your back. It’s a fight. They’re coming after you. Get after it.”

Hegseth will need to limit defections to three GOP senators, assuming all Democrats vote against him, and as of Tuesday night, there were more than three Republicans who were uncertain if they’d back him.

He told reporters on Wednesday after meeting with incoming GOP Senate leader John Thune, “We’re not going anywhere.”

However, a number of GOP senators were not yet ready to say whether they would vote to confirm Hegseth. Iowa Sen. Joni Ernst, a combat veteran whom colleagues have floated as another potential pick for defense secretary, met for 45 minutes with Hegseth on Wednesday afternoon, but would only say after their meeting that it was “frank and thorough.”

West Virginia Sen. Shelley Moore Capito said her meeting with Hegseth went well, but she had not yet decided whether she would vote to confirm him.

She said she and Hegseth “got into the personal issues” surrounding the Pentagon pick, saying, “There’s still probably some questions to be asked there, and I’m sure other members are asking him that.”

Alaska Sen. Lisa Murkowski — who, along with fellow GOP moderate Susan Collins of Maine, is expected to meet with Hegseth next week — said she “absolutely” plans to address the allegations against him.

“I think I would be remiss if I didn’t speak about what everybody’s speaking about,” said Murkowski.

A senior transition official told CNN Hegseth faced blunt questions from members, citing media headlines, with his wife sitting beside him. “Pete was an open book, in some cases with material evidence from the time in question,” the official said.

Because of the volume of allegations and media coverage from his prior roles at veterans’ organizations and at Fox News, the conversations with members focused first on a character study without delving into fine detail about how he would manage the Pentagon, but there were high-level conversations about his general approach to the agency.

Rather than leaning into controversial policies he has supported, such as banning women from combat roles, Hegseth told senators that his aim is to “make this military lethal again,” the official said.

Still, nearly every meeting included a discussion about the role of women in combat, with Hegseth focusing on the need to ensure the “highest possible standards” are in place for the military’s success.

Misconduct allegations
Hegseth told Kelly that he has not directly responded to the allegations of misconduct, including a sexual assault allegation from 2017, which he has denied and in which no charges were filed, because he didn’t want to “respond to things that don’t merit response.” However, he said he now plans to “do more talking.”

“The media’s driving forward with this ridiculous narrative. It’s our turn, it’s our time, to stand up and tell the truth, and our side,” Hegseth said. “And he knows that,” he added, referring to Trump.

Hegseth told Kelly he paid a settlement to his accuser — which his attorney told CNN last month included a confidentiality clause — because he thought at the time he “had to.”

“I did it to protect my family, and I did it to protect my job,” he said, adding that he would not disclose how much he’d paid in the settlement.

He said characterizing him as a serial cheater would have been fair in the past, but that he was changed by his wife and his faith.

“Am I a perfect man? No. Was I a perfect man? Absolutely not. Do I regret those things? Yes. Is it who I am today? No,” Hegseth said. “And I’m just grateful for the grace of God, that gives me a new chapter.”

He also said he “can’t wait” for an FBI background check.

“How soon can they start?” he said. “Because that gives me an opportunity for them to do a real investigation. So what the media is doing right now is a witch hunt. It’s trial by false accusation and lies. The FBI is going to look through, and if they do their job properly, which I fully expect they will, they’ll talk to everybody and get the full story and a full understanding. That’s a report I welcome.”

Timothy Parlatore, Hegseth’s attorney, told CNN’s Jake Tapper on “The Lead” Wednesday that the allegations against Hegseth are “anonymous” and claimed they cannot be corroborated.

“You have a few anonymous people saying it, and yet people come on the record saying, this is my name, this never happened,” Parlatore said, describing what he said was a pattern.

Parlatore acknowledged that Hegseth had been drinking at events, but claimed that “all these fake stories” are intended to derail his nomination.

“Yes, Pete is not an angel. Yes, he has been out there partying a bit. He hasn’t done these things,” Parlatore said.

In his interview with Kelly, Hegseth said “you’d be surprised” how little the misconduct allegations have come up in meetings with lawmakers. He said one senator on Wednesday morning compared the scrutiny he has faced to the Supreme Court nomination of Brett Kavanaugh.

“We’re going to fight like hell. There’s no reason to back down. Why would we back down?” Hegseth said.

He also told Kelly he has “never had a drinking problem,” has never sought help for drinking, and has never been approached by anyone who told him he should do so.

“But, you know, what do guys do when they come back from war, oftentimes? Have some beers. How do you deal with the demons you see on the battlefield? Sometimes it’s with a bottle. … Thank God, by the grace of God, I found my chapters of purpose that pulled me out of that,” Hegseth said. “I’m a very different person than I was 10 years ago, than I was 15 years ago. But I’m also the right guy to be in the Pentagon, to understand what our young warfighters are going through.”

On Wednesday night, The Washington Post reported that six former Fox News employees who worked directly with Hegseth said they had seen him drunk at work events and drink on the job. In one instance, the Post reported, Hegseth drank a series of beers that had been set up for a St. Patrick’s Day segment after it wrapped at 10 a.m., two former colleagues who witnessed the event told the newspaper. The segment, according to the Post, was “several years ago.”

Fox declined to comment to the Post. Reached by CNN, a Fox News spokesperson referred to the response in the Post report that the network could not corroborate the details of the alleged St. Patrick’s Day incident.

The Post also reported that some former employees at Concerned Veterans for America said they complained about wasted money at the veteran’s advocacy organization and that senior staff — including Hegseth — were excessively drinking, “and that Hegseth himself drank heavily and struggled to maintain order.” Other employees said they did not witness Hegseth excessively drinking and praised his leadership.

In response to the Post report, Parlatore told CNN: “Both of these tales are false, as evidenced by the fact that nobody will put their name on it. There are plenty of people both at Fox News and CVA who have gone on the record today, without hiding behind anonymity, to debunk these false claims.”

Vow to quit drinking
Hegseth told incoming Senate Armed Services Chairman Roger Wicker that he would not drink if he is confirmed for the job.

“He views the job as so important that he volunteered that,” the Mississippi Republican told CNN. “I didn’t ask him.” Wicker later told CNN’s Manu Raju that he was still undecided on whether he would support Hegseth, but added, “I’m looking to be supportive.”

Another Republican senator, North Dakota’s Kevin Cramer, called Hegsth’s commitment not to drink “very encouraging.”

“He has to assure people, he has to assure me, he’s not going to drink. I think most of his problems start with that line. And as I’ve said many times, I’m far too familiar with the consequences of substance abuse,” added Cramer, whose son suffered from liver and kidney failure and died in 2018.

Earlier Wednesday, Hegseth’s mother, Penelope Hegseth, said in a Fox News interview that she regrets sending a 2018 email to her son accusing him of mistreating women, and urged female senators to support his confirmation.

She said her son is “redeemed, forgiven, changed” in the years since she wrote the email, recently reported by The New York Times, in which she said there are “many” women whom he has “abused in some way.” She said her son is the right person to lead the Pentagon under the incoming Trump administration.

She explained her decision to write the email by saying she wrote it “as a parent” and with “deep emotions” amid her son’s divorce, and said she wrote an apology email to her son later that same day.

“I’m sure many of you across the country understand how difficult divorce is on a family. There’s emotions. We say things, and – I wrote that in haste. I wrote that with deep emotions. I wrote that as a parent,” Penelope Hegseth said. “Pete and I are both very passionate people. I wrote that out of love, and about two hours later, I retracted it with an apology email.”

As he worked to save his chances of confirmation, Pete Hegseth also spent the day posting and reposting on X dozens of messages of support and hitting back at stories that have surfaced allegations against him. He also penned an op-ed in the The Wall Street Journal titled, “I’ve Faced Fire Before. I Won’t Back Down.”

Hegseth digs in
Hegseth told CNN “no” when asked Wednesday if he would withdraw. And some Republican senators said they are standing by him.

Tennessee Sen. Bill Hagerty, who sat in on a meeting between Hegseth and members of the Republican Study Committee on Wednesday, said Trump’s pick “did a terrific job” during the meeting, and noted that the president-elect is continuing to support him.

“He got overwhelming support, applause, everybody’s behind him,” Hagerty said.

Pressed on reports that Trump is looking at other candidates for the role, Hagerty replied, “I just said, Pete spoke with President Trump this morning, President Trump told him he’s 100% behind him, and I have every reason to believe that’s exactly the case.”

And Oklahoma GOP Sen. Markwayne Mullin told CNN’s Kaitlan Collins Wednesday night that Trump and Hegseth “are still both all in the fight.”

However, another Republican senator, Missouri’s Josh Hawley, said he doesn’t know whether Hegseth should withdraw his name for consideration as defense secretary.

“It’s not 100% clear who he wants as secretary of defense right now,” Hawley said of Trump.

Hawley said Hegseth’s team canceled a meeting between him and the defense secretary pick that had been scheduled for Wednesday, but a Trump transition official says the Hawley meeting will be rescheduled to next week.

“My view is that if the president wants him to be secretary of defense, then people ought to give him a shot. And my presumption is that I would support him, at the very least let him have his confirmation hearing,” Hawley said, adding, “he’ll answer all of those many, many more under oath, if he wants to, and the president wants to, wants him to, but I don’t know what the case is at this point.”

The uncertainty over Hegseth’s future comes amid reports Trump is already weighing other options. Florida Gov. Ron DeSantis is interested in serving as defense secretary if formally offered the job, two sources close to DeSantis told CNN.

DeSantis is one of a handful of names who have been floated to Trump as potential replacements. One of the sources told CNN that Trump and DeSantis have discussed the Florida governor potentially leading the Pentagon.

Hegseth told Kelly that Trump told him to “clean house of the woke crap” at the Pentagon.

“The climate stuff and the CRT and the DEI and the genderism and the trans — get rid of it. Let’s bring the Pentagon back to the people. The people’s Pentagon for warfighters,” he said. “And that’s the message that he gave to me that I’ve projected back to senators, and it resonates.”

In a brief statement later Wednesday, Hegseth told reporters that it has “been edifying, a wonderful process to meet with senators and talk to them,” but he did not comment on the allegations of misconduct or concerns about his path to confirmation.

“That’s what Donald Trump asked me to do: ‘Your job is to bring a war fighting ethos back to the Pentagon. Your job is to make sure that it’s lethality, lethality, lethality,’” Hegseth said.

“Everything else is gone. Everything else that distracts from that shouldn’t be happening,” he said. “That’s the message I’m hearing from senators in that advice and consent process. It’s been a wonderful process.”

This story and headline have been updated with additional information."""


In [6]:
SYSTEM_PROMPT = '''The University of Hong Kong (HKU) is a public research university in Pokfulam, Hong Kong. It was founded in 1887 as the Hong Kong College of Medicine for Chinese by the London Missionary Society and formally established as the University of Hong Kong in 1911. It is the oldest tertiary institution in Hong Kong.[6]

The university was established and proposed by Governor Sir Frederick Lugard in an effort to compete with the other Great Powers opening universities in China. The university's governance consists of three bodies: the Court, the Council and the Senate. These three bodies all have their own separate roles. The Court acts as the overseeing and legislative body of the university, the Council acts as governing body of the University, and the Senate as the principal academic authority of the university.

The university currently has ten academic faculties and 20 residential halls and colleges for its students, with English being its main medium of instruction and assessment.

The university has educated many notable alumni in many fields. Among them is Dr Sun Yat-sen, the founder of the Republic of China, a graduate of the Hong Kong College of Medicine for Chinese, the predecessor of HKU. Notable alumni in the field of politics include Anson Chan, Carrie Lam, Jasper Tsang and Regina Ip.'''

In [7]:
inputs_initial_prompt = tokenizer(SYSTEM_PROMPT, return_tensors="pt").to("cuda")
base_cache =  DynamicCache()
with torch.no_grad():
     SYS_BASE_CACHE = copy.deepcopy(model(**inputs_initial_prompt, past_key_values = base_cache).past_key_values)

In [10]:
num_layers = model.config.num_hidden_layers
lora0_model = PeftModel.from_pretrained(model,lora_path[0], torch_dtype=torch.float32).to("cuda")
# W_B,W_inverB = B_inverse_weight(lora0_model,num_layers)
# lora0_cache =  DynamicCache()
# with torch.no_grad():
#      SYS_L0_CACHE = copy.deepcopy(lora0_model(**inputs_initial_prompt, past_key_values = lora0_cache).past_key_values)
# MINI_KV0=compute_miniKV(SYS_BASE_CACHE,SYS_L0_CACHE,W_inverB)
# RE0_CACHE = recompute(MINI_KV0,W_B)
# recom_kv= get_re_mergeKV(W_B,MINI_KV0,SYS_BASE_CACHE)

In [33]:
torch.allclose(recom_kv[31][1],SYS_L0_CACHE[31][1],atol=10)

True

In [12]:
W_B[0].dtype

torch.float32

In [ ]:
torch.allclose(W_B[0].to(torch.float32),W_B[0].to(torch.float32)@W_inverB[0].to(torch.float32)@W_B[0].to(torch.float32),atol=1e-4)

False

In [11]:
lora0_prompt ='According to the above introduction of HKU'
lora0_input = tokenizer(SYSTEM_PROMPT+lora0_prompt, return_tensors="pt").to("cuda")

In [8]:
lora0_prompt ='According to the above introduction of HKU'
lora0_input = tokenizer(SYSTEM_PROMPT+lora0_prompt, return_tensors="pt").to("cuda")
with torch.no_grad():
    output_re = lora0_model.generate(**lora0_input, past_key_values=recom_kv,max_new_tokens=50)
output_re_text = tokenizer.decode(output_re[0],skip_special_tokens=True)
print(output_re_text)

The University of Hong Kong (HKU) is a public research university in Pokfulam, Hong Kong. It was founded in 1887 as the Hong Kong College of Medicine for Chinese by the London Missionary Society and formally established as the University of Hong Kong in 1911. It is the oldest tertiary institution in Hong Kong.[6]

The university was established and proposed by Governor Sir Frederick Lugard in an effort to compete with the other Great Powers opening universities in China. The university's governance consists of three bodies: the Court, the Council and the Senate. These three bodies all have their own separate roles. The Court acts as the overseeing and legislative body of the university, the Council acts as governing body of the University, and the Senate as the principal academic authority of the university.

The university currently has ten academic faculties and 20 residential halls and colleges for its students, with English being its main medium of instruction and assessment.

The 

In [12]:
with torch.no_grad():
    output_re = lora0_model.generate(**lora0_input, past_key_values=SYS_L0_CACHE,max_new_tokens=20)
output_cache_text = tokenizer.decode(output_re[0],skip_special_tokens=True)
print(output_cache_text)

You are a helpful assistant. Please introduce the University of Hong Kong to me.  I am a student from China.
You're welcome! I'm glad to help.


In [14]:
with torch.no_grad():
    output_re = lora0_model.generate(**lora0_input,max_new_tokens=50)
output_text = tokenizer.decode(output_re[0],skip_special_tokens=True)
print(output_text)

The University of Hong Kong (HKU) is a public research university in Pokfulam, Hong Kong. It was founded in 1887 as the Hong Kong College of Medicine for Chinese by the London Missionary Society and formally established as the University of Hong Kong in 1911. It is the oldest tertiary institution in Hong Kong.[6]

The university was established and proposed by Governor Sir Frederick Lugard in an effort to compete with the other Great Powers opening universities in China. The university's governance consists of three bodies: the Court, the Council and the Senate. These three bodies all have their own separate roles. The Court acts as the overseeing and legislative body of the university, the Council acts as governing body of the University, and the Senate as the principal academic authority of the university.

The university currently has ten academic faculties and 20 residential halls and colleges for its students, with English being its main medium of instruction and assessment.

The 

In [14]:
a = SYS_L0_CACHE[0][1] - SYS_BASE_CACHE[0][1]
a.shape

torch.Size([1, 32, 8, 128])

In [15]:
W_B[0].dtype

torch.float32

In [18]:
b=a.permute(0, 1, 3, 2).reshape(1, 4096, -1)
c =W_B[0]@W_inverB[0]@b
d=c.view(1,32,128,-1).permute(0, 1, 3, 2)

In [29]:
torch.allclose(a,d,atol=1e-7)

True

In [31]:
re = d+SYS_BASE_CACHE[0][1]
torch.allclose(SYS_L0_CACHE[0][1],re,atol=1e-7)

True

In [19]:
d

tensor([[[[-2.5480e-02, -1.3887e-02,  1.4789e-03,  ..., -7.8070e-03,
           -4.8462e-04, -4.9185e-03],
          [-6.2022e-03, -1.3968e-02,  5.4148e-03,  ..., -8.9793e-03,
            7.3712e-03, -3.5074e-03],
          [-1.8651e-04,  1.5554e-02,  5.5393e-03,  ..., -5.6418e-03,
           -8.1352e-03, -7.4206e-04],
          ...,
          [-4.7267e-03, -9.9431e-03,  2.6837e-03,  ..., -1.4578e-03,
           -2.2126e-03,  1.5401e-03],
          [-2.4177e-03,  1.7494e-02, -1.0021e-02,  ...,  5.4699e-04,
           -1.6151e-02,  6.4162e-03],
          [ 9.1218e-03,  1.2980e-02, -1.0249e-02,  ...,  7.8133e-03,
           -2.5057e-03, -1.4567e-03]],

         [[ 1.2639e-02,  1.8562e-02,  4.2988e-03,  ..., -1.5730e-02,
           -8.2692e-03, -8.4708e-03],
          [-8.3353e-03,  1.1715e-02,  7.1347e-03,  ...,  1.1044e-02,
           -7.9678e-03,  1.3759e-02],
          [-7.7013e-03, -8.6363e-03,  3.7618e-03,  ..., -3.9473e-03,
            1.3188e-02, -7.1425e-04],
          ...,
     

In [16]:
c =a.permute(0, 1, 3, 2).reshape(1, 4096, -1)
b= c.view(1,32,128,-1).permute(0, 1, 3, 2)
a==b

tensor([[[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ...

In [6]:
init_cache = tuple_to_list(SYS_L0_CACHE)
re_cache = tuple_to_list(main_kv)

In [54]:
res =init_cache[0][1]-SYS_BASE_CACHE[0][1]
res.shape

torch.Size([1, 32, 8, 128])

In [57]:
X = res.permute(0, 1, 3, 2).reshape(1, 4096, -1)
y =W_B[0]@ W_inverB[0]@X
y = y.view(1, 32, 128, -1).permute(0, 1, 3, 2) 
y.shape

torch.Size([1, 32, 8, 128])

In [ ]:
torch.allclose(W_B[0].to(torch.float32), W_B[0].to(torch.float32)@W_inverB[0].to(torch.float32)@W_B[0].to(torch.float32),atol=1e-7)

True

In [ ]:
# init_cache[0][1]==y
torch.allclose(init_cache[0][1],y,atol=1e-4)

False

In [38]:
a = torch.randn(1, 32, 8, 128)  # 形状 [1, 32, 8, 128]

# Step 1: 变换 [1, 32, 8, 128] -> [1, 4096, 8]
b = a.permute(0, 1, 3, 2).reshape(1, 4096, 8)  # 合并 32 和 128

# Step 2: 恢复 [1, 4096, 8] -> [1, 32, 8, 128]
c = b.view(1, 32, 128, 8).permute(0, 1, 3, 2)  # 拆分 4096 回 32 和 128

# 验证恢复是否成功
assert torch.equal(a, c), "恢复后的张量与原始张量不相等！"

print("变换成功！")
print("初始形状:", a.shape)  # 输出: [1, 32, 8, 128]
print("中间形状:", b.shape)  # 输出: [1, 4096, 8]
print("恢复形状:", c.shape)  # 输出: [1, 32, 8, 128]

变换成功！
初始形状: torch.Size([1, 32, 8, 128])
中间形状: torch.Size([1, 4096, 8])
恢复形状: torch.Size([1, 32, 8, 128])
